In [3]:
import numpy
import pandas

import json
import connalysis

from matplotlib import pyplot as plt

from helpers import network, plotting
from helpers import analysis
from helpers.bootstrap import save_data

In [60]:
# HELPER FUNCTION TO COUNT INTERACTIONS BETWEEN INH. NEURONS AND SIMPLICES
# FOR DIFFERENT CLASSES OF INH. NEURONS
def innervation_prob_by_status(paths_in, col_status):
    path_cp = paths_in.copy()
    idx = path_cp.index.to_frame()
    inh_idx = idx["Inhibitory"].reset_index(drop=True)
    inh_stats = Mnrn.vertices.loc[inh_idx][[col_status]]

    idx = pandas.MultiIndex.from_frame(pandas.concat([idx.reset_index(drop=True),
                                           inh_stats.reset_index(drop=True)], axis=1))
    path_cp.index = idx
    
    res = path_cp.groupby(["Position", "Simplex", col_status]).sum().unstack(col_status, fill_value=0)

    res = (res / Mnrn.vertices[col_status].value_counts()).fillna(0)
    res = res.unstack("Position")
    return res

In [87]:
for i in range(15):
    fn_in = "configs/microns1181_placement_{0}.json".format(i)
    with open(fn_in, "r") as fid:
        cfg = json.load(fid)
    plot_out = cfg["plotting"]["output_root"]
    save_out = cfg["save"]["filename"]
    save_id = cfg["save"]["id"]
    
    #LOAD
    M = network.load_network(cfg["connectome"]["loading"])
    
    Msmpl = network.filter_network(M, cfg["connectome"]["subnetwork"]["simplices"])
    name_nrn = cfg["connectome"]["subnetwork"]["neurons"]["name"]
    Mnrn = network.filter_network(M, cfg["connectome"]["subnetwork"]["neurons"]["filters"])

    print(len(Msmpl))
    print(len(Mnrn))
    
    # FIND SIMPLICES
    dimensions = [cfg["analysis"]["simplex_dimension"]]

    simplices, df_gids, df_idxs, ugids = analysis.get_simplex_dataframes(Msmpl, dimensions)
    ugid = ugids[0]
    df_gid = df_gids[0]
    df_idx = df_idxs[0]
    tgt_dim = dimensions[0]
    
    # PROBABILITY THAT NEURONS PARTICIPATE IN A GIVEN SIMPLEX POSITION
    _bins = numpy.hstack([Msmpl.gids, 1E12])
    part_in_positions = df_gid.apply(lambda _x: numpy.histogram(_x, bins=_bins)[0] > 0, axis=0)
    for pr_col in ["status_axon", "status_dendrite"]:
        part_in_positions[pr_col] = Msmpl.vertices[pr_col].values
    save_data(part_in_positions, save_out, save_id, "prob_participation_in_position")
    
    # FIND INH-SMPL-INH PATHS
    m_nrn_smpl, m_smpl_nrn = analysis.exc_inh_sparce_matrices(M, ugid, Mnrn.gids)
    s_n_paths, n_s_paths = analysis.get_simplex_neuron_path_df(m_nrn_smpl, m_smpl_nrn, df_idx, name_nrn)
    
    # NODE PARTICIPATION
    np = connalysis.network.topology.node_participation(Msmpl.matrix)
    np["status_dendrite"] = Msmpl.vertices["status_dendrite"].values
    np["status_axon"] = Msmpl.vertices["status_axon"].values
    
    smpl_counts_vld_dend = np.groupby(["status_dendrite"])[[1,2,3,4,5,6]].mean()
    save_data(smpl_counts_vld_dend, save_out, save_id, "smpl_counts_vld_dend")
    
    smpl_counts_vld_axon = np.groupby(["status_axon"])[[1,2,3,4,5,6]].mean()
    save_data(smpl_counts_vld_axon, save_out, save_id, "smpl_counts_vld_axon")
    
    # INTERACTION OF INH NEURONS WITH SIMPLICES FOR DIFFERENT CLASSES OF NEURONS
    ## 1. FOR VALIDATED VS NON-VALIDATED NEURONS
    inn_by_validation = pandas.concat([
        innervation_prob_by_status(s_n_paths, "status_dendrite"),
        innervation_prob_by_status(n_s_paths, "status_axon")
    ], keys=["Smpl-to-inh", "Inh-to-smpl"], names=["direction"], axis=1)
    save_data(inn_by_validation, save_out, save_id, "inh_interactions_by_proofread")
    
    ## 2. BY INHIBITORY TYPES
    inn_by_class = pandas.concat([
        innervation_prob_by_status(s_n_paths, "cell_type"),
        innervation_prob_by_status(n_s_paths, "cell_type")
    ], keys=["Smpl-to-inh", "Inh-to-smpl"], names=["direction"], axis=1)
    save_data(inn_by_class, save_out, save_id, "inh_interactions_by_cell_type")
    
    # FOR COMPARISON: DEGREES OF INH TYPES
    m_e_i = M.submatrix(Msmpl.gids, sub_gids_post=Mnrn.gids) > 0
    e_indeg = numpy.array(m_e_i.sum(axis=0))[0]

    m_i_e = M.submatrix(Mnrn.gids, sub_gids_post=Msmpl.gids) > 0
    e_outdeg = numpy.array(m_i_e.sum(axis=1))[:, 0]

    df = pandas.concat([
        Mnrn.vertices[["cell_type"]],
        pandas.DataFrame({"e_indeg": e_indeg, "e_outdeg": e_outdeg})
    ], axis=1)
    e_degrees_by_class = pandas.concat([df.groupby("cell_type").mean(),
                                        df["cell_type"].value_counts()],
                                       axis=1)
    save_data(e_degrees_by_class, save_out, save_id, "e_degrees_by_inh_class")
    
    # WHICH INHIBITORY NEURONS HAVE BEEN VALIDATED?
    vld_stats = pandas.concat([
    Mnrn.vertices["cell_type"].value_counts(),
    Mnrn.vertices.groupby("cell_type")["status_dendrite"].apply(lambda _x: (_x == "t").sum()),
    Mnrn.vertices.groupby("cell_type")["status_axon"].apply(lambda _x: (_x == "t").sum()),
    Mnrn.vertices.groupby("cell_type")["newly_valid"].sum()
    ], axis=1)

    save_data(vld_stats, save_out, save_id, "inh_validation_stats")


18756
7887


100%|██████████| 1/1 [00:00<00:00, 200.28it/s]
/gpfs/bbp.cscs.ch/home/reimann/venvs/bpsnap/lib/python3.10/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '0'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  df.to_hdf(filename, key="{0}/{1}".format(dset, data_id))
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferre

18724
7887


100%|██████████| 1/1 [00:00<00:00, 170.78it/s]
/gpfs/bbp.cscs.ch/home/reimann/venvs/bpsnap/lib/python3.10/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  df.to_hdf(filename, key="{0}/{1}".format(dset, data_id))
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferre

18237
7887


...

100%|██████████| 1/1 [00:00<00:00, 225.25it/s]
/gpfs/bbp.cscs.ch/home/reimann/venvs/bpsnap/lib/python3.10/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '12'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  df.to_hdf(filename, key="{0}/{1}".format(dset, data_id))
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferr

16964
7887


100%|██████████| 1/1 [00:00<00:00, 203.10it/s]
/gpfs/bbp.cscs.ch/home/reimann/venvs/bpsnap/lib/python3.10/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '13'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  df.to_hdf(filename, key="{0}/{1}".format(dset, data_id))
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferr

16609
7887


100%|██████████| 1/1 [00:00<00:00, 197.43it/s]
/gpfs/bbp.cscs.ch/home/reimann/venvs/bpsnap/lib/python3.10/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '14'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  df.to_hdf(filename, key="{0}/{1}".format(dset, data_id))
/gpfs/bbp.cscs.ch/home/reimann/code/notebooks/excitation_inhibition_topology_github/helpers/bootstrap.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferr